In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [10]:
df = pd.read_csv("02_Test_Data_Set/card_cust.csv")
df.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


In [11]:
df.isna().sum()

CUST_ID                              0
BALANCE                              0
BALANCE_FREQUENCY                    0
PURCHASES                            0
ONEOFF_PURCHASES                     0
INSTALLMENTS_PURCHASES               0
CASH_ADVANCE                         0
PURCHASES_FREQUENCY                  0
ONEOFF_PURCHASES_FREQUENCY           0
PURCHASES_INSTALLMENTS_FREQUENCY     0
CASH_ADVANCE_FREQUENCY               0
CASH_ADVANCE_TRX                     0
PURCHASES_TRX                        0
CREDIT_LIMIT                         0
PAYMENTS                             0
MINIMUM_PAYMENTS                    74
PRC_FULL_PAYMENT                     0
TENURE                               0
dtype: int64

In [13]:
# 전처리
base = df.copy()
base["CREDIT_LIMIT"] = base["CREDIT_LIMIT"].fillna(base["CREDIT_LIMIT"].mean())
base["MINIMUM_PAYMENTS"] = base["MINIMUM_PAYMENTS"].fillna(base["MINIMUM_PAYMENTS"].mean())
base.isna().sum()

CUST_ID                             0
BALANCE                             0
BALANCE_FREQUENCY                   0
PURCHASES                           0
ONEOFF_PURCHASES                    0
INSTALLMENTS_PURCHASES              0
CASH_ADVANCE                        0
PURCHASES_FREQUENCY                 0
ONEOFF_PURCHASES_FREQUENCY          0
PURCHASES_INSTALLMENTS_FREQUENCY    0
CASH_ADVANCE_FREQUENCY              0
CASH_ADVANCE_TRX                    0
PURCHASES_TRX                       0
CREDIT_LIMIT                        0
PAYMENTS                            0
MINIMUM_PAYMENTS                    0
PRC_FULL_PAYMENT                    0
TENURE                              0
dtype: int64

## Q1.

In [43]:
base_q1 = base[["TENURE", "BALANCE", "CREDIT_LIMIT"]]
base_q1

,TENURE,BALANCE,CREDIT_LIMIT
0,12.0,40.900749,1000.0
1,12.0,3202.467416,7000.0
2,12.0,2495.148862,7500.0
3,12.0,1666.670542,7500.0
4,12.0,817.714335,1200.0
...,...,...,...
995,11.0,1250.394614,3500.0
996,12.0,9.503968,4500.0
997,12.0,2285.068731,2500.0
998,12.0,2928.756699,3000.0


In [92]:
base_q1["TENURE"].unique()

array([12.,  8., 11.,  9., 10.,  7.,  6.])

In [45]:
base_q1["TENURE"].value_counts()

12.0    898
11.0     35
8.0      18
10.0     18
9.0      11
6.0      11
7.0       9
Name: TENURE, dtype: int64

In [46]:
base_q1.groupby("TENURE")[["BALANCE", "CREDIT_LIMIT"]].corr()

BALANCE  CREDIT_LIMIT
TENURE                                     
6.0    BALANCE       1.000000      0.868056
       CREDIT_LIMIT  0.868056      1.000000
7.0    BALANCE       1.000000      0.948405
       CREDIT_LIMIT  0.948405      1.000000
8.0    BALANCE       1.000000      0.820696
       CREDIT_LIMIT  0.820696      1.000000
9.0    BALANCE       1.000000      0.085474
       CREDIT_LIMIT  0.085474      1.000000
10.0   BALANCE       1.000000      0.291482
       CREDIT_LIMIT  0.291482      1.000000
11.0   BALANCE       1.000000      0.380360
       CREDIT_LIMIT  0.380360      1.000000
12.0   BALANCE       1.000000      0.460833
       CREDIT_LIMIT  0.460833      1.000000

In [47]:
round(0.948405, 2)

0.95

## Q2.

In [20]:
base.drop(labels = "CUST_ID", axis=1)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.000000,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.250000,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0
2,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,12.0,7500.0,622.066742,627.284787,0.000000,12.0
3,1666.670542,0.636364,1499.00,1499.00,0.00,205.788017,0.083333,0.083333,0.000000,0.083333,1.0,1.0,7500.0,0.000000,1297.116322,0.000000,12.0
4,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0.0,1.0,1200.0,678.334763,244.791237,0.000000,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1250.394614,0.909091,443.99,443.99,0.00,0.000000,0.272727,0.272727,0.000000,0.000000,0.0,3.0,3500.0,273.823646,259.715939,0.000000,11.0
996,9.503968,1.000000,96.62,0.00,96.62,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,19.0,4500.0,1086.932525,92.217936,0.250000,12.0
997,2285.068731,1.000000,0.00,0.00,0.00,1173.310874,0.000000,0.000000,0.000000,0.166667,5.0,0.0,2500.0,381.672065,1003.265207,0.000000,12.0
998,2928.756699,1.000000,160.92,0.00,160.92,319.931964,1.000000,0.000000,1.000000,0.333333,5.0,12.0,3000.0,1142.847203,1098.479834,0.000000,12.0


In [22]:
df_q2_st = StandardScaler().fit_transform(base.drop(labels = "CUST_ID", axis=1))
df_q2_st

array([[-0.84876759, -0.41987944, -0.4419358 , ..., -0.44372465,
        -0.46554357,  0.28242902],
       [ 0.28279099,  0.01213096, -0.4690169 , ..., -0.08615941,
         0.33159169,  0.28242902],
       [ 0.0296341 ,  0.44414137, -0.24953794, ..., -0.25675456,
        -0.46554357,  0.28242902],
       ...,
       [-0.04555583,  0.44414137, -0.4690169 , ..., -0.1126367 ,
        -0.46554357,  0.28242902],
       [ 0.18482699,  0.44414137, -0.4233367 , ..., -0.07613978,
        -0.46554357,  0.28242902],
       [-0.34079285,  0.44414137, -0.20275918, ...,  0.00909944,
        -0.46554357,  0.28242902]])

In [32]:
score_list = []
k_list = list(range(2, 6))
model_list = []
i = 0
for k in k_list:
    model_list.append(KMeans(n_clusters = k, random_state= 1234).fit(df_q2_st))
    score_list.append(silhouette_score(X = df_q2_st, labels= model_list[i].labels_))
    i += 1
# 실루엣 스코어의 random_state는 sample 사이즈를 모를때 쓰는 것. 여기에선 전체 데이터셋을 다 쓰기 때문에 해당 X
idx = score_list.index(max(score_list))
idx, k_list[idx]

(0, 2)

In [40]:
# model_list[0].labels_
# type(model_list[0].labels_)
df_q2 = base.copy()
df_q2["cluster"] = pd.DataFrame(model_list[0].labels_, index = df_q2.index)
df_q2.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,cluster
0,10001,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0,0
1,10002,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0,0


In [41]:
df_q2.groupby("cluster")["ONEOFF_PURCHASES"].mean()

cluster
0     340.230998
1    3946.187525
Name: ONEOFF_PURCHASES, dtype: float64

In [42]:
round(3946.187525, 2)

3946.19

## Q3.

In [72]:
df_tr = base.loc[base["CUST_ID"] % 4 != 0, :].drop(labels = "CUST_ID", axis = 1)
df_t = base.loc[base["CUST_ID"] % 4 == 0, :].drop(labels = "CUST_ID", axis = 1)

In [73]:
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   BALANCE                           752 non-null    float64
 1   BALANCE_FREQUENCY                 752 non-null    float64
 2   PURCHASES                         752 non-null    float64
 3   ONEOFF_PURCHASES                  752 non-null    float64
 4   INSTALLMENTS_PURCHASES            752 non-null    float64
 5   CASH_ADVANCE                      752 non-null    float64
 6   PURCHASES_FREQUENCY               752 non-null    float64
 7   ONEOFF_PURCHASES_FREQUENCY        752 non-null    float64
 8   PURCHASES_INSTALLMENTS_FREQUENCY  752 non-null    float64
 9   CASH_ADVANCE_FREQUENCY            752 non-null    float64
 10  CASH_ADVANCE_TRX                  752 non-null    float64
 11  PURCHASES_TRX                     752 non-null    float64
 12  CREDIT_L

In [74]:
df_tr = df_tr.set_index("ONEOFF_PURCHASES").reset_index()
df_t = df_t.set_index("ONEOFF_PURCHASES").reset_index()
df_tr.head(2)

,ONEOFF_PURCHASES,BALANCE,BALANCE_FREQUENCY,PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,0.0,40.900749,0.818182,95.4,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,0.0,3202.467416,0.909091,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


In [76]:
model_tree = DecisionTreeRegressor(random_state=1234)\
                .fit(X = df_tr.iloc[:, 1:], y = df_tr["ONEOFF_PURCHASES"])

In [78]:
pred = model_tree.predict(df_t.iloc[:, 1:])
pred[:5]

array([1500.,    0., 1490.,    0.,    0.])

In [79]:
from sklearn.metrics import mean_squared_error

In [81]:
round(mean_squared_error(y_true = df_t["ONEOFF_PURCHASES"], y_pred = pred) ** 0.5, 1)

1039.2

In [83]:
a = base.iloc[:10, :3]
a

,CUST_ID,BALANCE,BALANCE_FREQUENCY
0,10001,40.900749,0.818182
1,10002,3202.467416,0.909091
2,10003,2495.148862,1.000000
3,10004,1666.670542,0.636364
4,10005,817.714335,1.000000
5,10006,1809.828751,1.000000
6,10007,627.260806,1.000000
7,10008,1823.652743,1.000000
8,10009,1014.926473,1.000000
9,10010,152.225975,0.545455


In [86]:
b = base.iloc[:10, 12:16]
b

,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS
0,2.0,1000.0,201.802084,139.509787
1,0.0,7000.0,4103.032597,1072.340217
2,12.0,7500.0,622.066742,627.284787
3,1.0,7500.0,0.000000,1297.116322
4,1.0,1200.0,678.334763,244.791237
5,8.0,1800.0,1400.057770,2407.246035
6,64.0,13500.0,6354.314328,198.065894
7,12.0,2300.0,679.065082,532.033990
8,5.0,7000.0,688.278568,311.963409
9,3.0,11000.0,1164.770591,100.302262


In [91]:
pd.concat([a, b], axis = 1)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS
0,10001,40.900749,0.818182,2.0,1000.0,201.802084,139.509787
1,10002,3202.467416,0.909091,0.0,7000.0,4103.032597,1072.340217
2,10003,2495.148862,1.000000,12.0,7500.0,622.066742,627.284787
3,10004,1666.670542,0.636364,1.0,7500.0,0.000000,1297.116322
4,10005,817.714335,1.000000,1.0,1200.0,678.334763,244.791237
5,10006,1809.828751,1.000000,8.0,1800.0,1400.057770,2407.246035
6,10007,627.260806,1.000000,64.0,13500.0,6354.314328,198.065894
7,10008,1823.652743,1.000000,12.0,2300.0,679.065082,532.033990
8,10009,1014.926473,1.000000,5.0,7000.0,688.278568,311.963409
9,10010,152.225975,0.545455,3.0,11000.0,1164.770591,100.302262


In [95]:
a.drop_duplicates(subset="BALANCE_FREQUENCY")

,CUST_ID,BALANCE,BALANCE_FREQUENCY
0,10001,40.900749,0.818182
1,10002,3202.467416,0.909091
2,10003,2495.148862,1.000000
3,10004,1666.670542,0.636364
9,10010,152.225975,0.545455
